In [11]:
# all imports
import numpy as np
import h5py
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd

In [ ]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/'
v_dir = '/pscratch/sd/j/jjpim/massmag/'

# load updated master table
tf_mastertable = Table.read(data_dir + 'tf_mastertable_massmag_stellar.fits') 

#load SGA table
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

In [13]:
tf_mastertable[tf_mastertable['TARGETID']==39633220467166633]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,Z_CENTER,ROTCURVE_CHI2,VMAX,VMAX_ERR,RTURN,RTURN_ERR,ALPHA,ALPHA_ERR,INCLINATION,VROT_R26,VROT_R26_ERR,M_R26,M_R26_ERR,VROT_3P5R26,VROT_3P5R26_ERR,M_3P5R26,M_3P5R26_ERR,RABS_MAG26,M_STELLAR_CIGALE,M_STELLAR_CIGALE_ERR,M_STELLAR_PROVABGS,M_STELLAR_PROVABGS_ERR
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39633220467166633,195.309207793123,48.2167412541527,11049,main,0.029156232349159,6.77173077873817e-06,0,1902.32760119066,iron/healpix/main/bright/110/11049/redrock-main-bright-11049.fits,TFT,857718,195.3092077931228,48.21674125415274,0.0,0.0,0.0,0.0,0.0,0.0,2.8710144904360484,0.029156232349159,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


In [ ]:
# Define the directory containing the data
prova_data_dir = '/global/cfs/cdirs/desi/users/chahah/sedflow/desiy1_bgs/'

# List all files in the directory
files = [os.path.join(prova_data_dir, file) for file in os.listdir(prova_data_dir)]

# Print out the full paths of the files
for file_path in files:
    print(file_path)
    
# Confirm the number of files
print(f"Total number of files: {len(files)}")  

In [ ]:
t1 = Table.read(prova_data_dir + 'BGS_ANY_full.vagc.83.hdf5') 

In [ ]:
from astropy.table import Table, join
filtered_table = join(tf_mastertable, t1, keys='TARGETID', join_type='inner')
print(filtered_table)

In [ ]:
'''def process_file(file_path, tf_mastertable):
    try:
        provabgs = Table.read(file_path, format='hdf5')
        
        #check for needed column
        if 'SEDFLOW_LOGMSTAR_MAP' not in provabgs.colnames:
            print(f"Warning: Column 'SEDFLOW_LOGMSTAR_MAP' not found in {file_path}. Skipping this file.")
            return tf_mastertable, True  # Return True to indicate this file was skipped

        #get target ids
        target_ids = tf_mastertable['TARGETID'][tf_mastertable['M_R26'] > 0]

        #find matching TARGETIDs in the provabgs table
        valid_targets_mask = np.isin(provabgs['TARGETID'], target_ids)
        matching_targets = provabgs[valid_targets_mask]

        if len(matching_targets) == 0:
            print(f"No matching targets found in {file_path}. Skipping.")
            return tf_mastertable, True  # Return True to indicate this file was skipped

        #update tf_mastertable
        for row in matching_targets:
            target_id = row['TARGETID']
            stellar_mass = row['SEDFLOW_LOGMSTAR_MAP']
            log_stellar_mass_samples = row['SEDFLOW_LOGMSTAR_SAMPLES']
            stellar_mass_samples_linear = 10 ** log_stellar_mass_samples

            std_linear = np.std(stellar_mass_samples_linear)
            mean_linear = np.mean(stellar_mass_samples_linear)
            std_log = std_linear / mean_linear

            #find the corresponding sga id for the target
            sga_id = tf_mastertable[tf_mastertable['TARGETID'] == target_id]['SGA_ID'][0]

            #update the M_STELLAR_PROVABGS and M_STELLAR_PROVABGS_ERR columns
            mask = np.logical_and(tf_mastertable['SGA_ID'] == sga_id, tf_mastertable['M_R26'] > 0)
            tf_mastertable['M_STELLAR_PROVABGS'][mask] = stellar_mass
            tf_mastertable['M_STELLAR_PROVABGS_ERR'][mask] = std_log

        return tf_mastertable, False  #not skipped

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return tf_mastertable, True  #skipped

In [ ]:
def update_stellar_masses(tf_mastertable, provabgs):
    # initialize list
    targets = []
    
    # Extract target IDs from tf_mastertable where M_R26 > 0
    target_ids = tf_mastertable['TARGETID'][tf_mastertable['M_R26'] > 0]
    
    # Find the matching targets in provabgs
    for target_id in target_ids:
        if len(provabgs[provabgs['TARGETID'] == target_id]) > 0:
            targets.append(target_id)
    
    # Loop through each matching target ID to extract and update masses
    for target_id in targets:
        # Extract stellar mass and log mass samples from provabgs
        stellar_mass = provabgs[provabgs['TARGETID'] == target_id]['SEDFLOW_LOGMSTAR_MAP'][0]
        log_stellar_mass_samples = provabgs[provabgs['TARGETID'] == target_id]['SEDFLOW_LOGMSTAR_SAMPLES'].data[0]
        
        # Convert to linear scale
        stellar_mass_samples_linear = 10 ** log_stellar_mass_samples
        
        # Calculate the standard deviation and mean in linear space
        std_linear = np.std(stellar_mass_samples_linear)
        mean_linear = np.mean(stellar_mass_samples_linear)
        
        # Calculate standard deviation in log space
        std_log = std_linear / mean_linear
        
        # Find corresponding SGA_ID
        sga_id = tf_mastertable[tf_mastertable['TARGETID'] == target_id]['SGA_ID'][0]
        
        # Update the tf_mastertable with stellar mass and error values
        tf_mastertable['M_STELLAR_PROVABGS'][np.logical_and(tf_mastertable['SGA_ID'] == sga_id, tf_mastertable['M_R26'] > 0)] = stellar_mass
        tf_mastertable['M_STELLAR_PROVABGS_ERR'][np.logical_and(tf_mastertable['SGA_ID'] == sga_id, tf_mastertable['M_R26'] > 0)] = std_log

    return tf_mastertable

In [ ]:
import os
import h5py
from astropy.table import Table

# Initialize the list of files
files = [os.path.join(prova_data_dir, file) for file in os.listdir(prova_data_dir)]

# Load the master table once (assuming it's the same for all files)
tf_mastertable = Table.read(data_dir + 'tf_mastertable_massmag_stellar.fits')  # Adjust the path as necessary

# Iterate through each file in the directory
for file in files:
    try:
        # Open the HDF5 file containing the Astropy table
        with h5py.File(file, 'r') as f:
            # List all available datasets in the file
            keys = list(f.keys())

            # Loop through the datasets to find the one that matches the structure of the Astropy table
            for key in keys:
                data = f[key][:]
                # Check if the data matches the structure of an Astropy table (e.g., 2D array, columns)
                if isinstance(data, (list, np.ndarray)):  # Adjust this check if needed
                    # Assuming that this is the right table structure, convert it into an Astropy table
                    provabgs = Table(data)  # Convert to Astropy Table
                    break
            else:
                raise KeyError(f"No matching dataset found in {file}")

        # Update tf_mastertable with stellar mass information
        tf_mastertable = update_stellar_masses(tf_mastertable, provabgs)

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Optionally, save the updated tf_mastertable after processing all files
#tf_mastertable.write('updated_tf_mastertable.fits', format='fits', overwrite=True)


In [25]:
def update_stellar_masses(tf_mastertable, provabgs):
    # Initialize columns if they don't exist
    if 'M_STELLAR_PROVABGS' not in tf_mastertable.colnames:
        tf_mastertable['M_STELLAR_PROVABGS'] = np.zeros(len(tf_mastertable))
    if 'M_STELLAR_PROVABGS_ERR' not in tf_mastertable.colnames:
        tf_mastertable['M_STELLAR_PROVABGS_ERR'] = np.zeros(len(tf_mastertable))
    
    # Initialize list
    targets = []
    
    # Extract target IDs from tf_mastertable where M_R26 > 0
    target_ids = tf_mastertable['TARGETID'][tf_mastertable['M_R26']]
    
    # Find the matching targets in provabgs
    for target_id in target_ids:
        if len(provabgs[provabgs['TARGETID'] == target_id]) > 0:
            targets.append(target_id)
    
    # Loop through each matching target ID to extract and update masses
    for target_id in targets:
        # Extract matching rows for the target_id in provabgs
        matching_rows = provabgs[provabgs['TARGETID'] == target_id]
        
        if len(matching_rows) > 0:
            # Extract stellar mass and log mass samples from provabgs
            stellar_mass = matching_rows['SEDFLOW_LOGMSTAR_MAP'][0]
            log_stellar_mass_samples = matching_rows['SEDFLOW_LOGMSTAR_SAMPLES'].data[0]
            
            # Convert to linear scale
            stellar_mass_samples_linear = 10 ** log_stellar_mass_samples
            
            # Calculate the standard deviation and mean in linear space
            std_linear = np.std(stellar_mass_samples_linear)
            mean_linear = np.mean(stellar_mass_samples_linear)
            
            # Calculate standard deviation in log space
            std_log = std_linear / mean_linear
            
            # Find corresponding SGA_ID
            sga_id = tf_mastertable[tf_mastertable['TARGETID'] == target_id]['SGA_ID'][0]
            
            # Update the tf_mastertable with stellar mass and error values
            mask = np.logical_and(tf_mastertable['SGA_ID'] == sga_id, tf_mastertable['M_R26'] > 0)
            tf_mastertable['M_STELLAR_PROVABGS'][mask] = stellar_mass
            tf_mastertable['M_STELLAR_PROVABGS_ERR'][mask] = std_log
            
            # Debugging print statements
            print(f"Updated target_id {target_id} (SGA_ID: {sga_id}):")
            print(f"  Stellar mass: {stellar_mass}")
            print(f"  Log mass samples: {log_stellar_mass_samples}")
            print(f"  Std in log space: {std_log}")
        else:
            print(f"No matching rows found for target_id {target_id} in provabgs.")

    # Return the updated tf_mastertable
    return tf_mastertable

# Initialize the list of files
files = [os.path.join(prova_data_dir, file) for file in os.listdir(prova_data_dir)]

# Load the master table once (assuming it's the same for all files)
tf_mastertable = Table.read(data_dir + 'tf_mastertable_massmag_stellar.fits')  # Adjust the path as necessary

# Iterate through each file in the directory
for file in files:
    try:
        # Open the HDF5 file containing the Astropy table
        with h5py.File(file, 'r') as f:
            # List all available datasets in the file
            keys = list(f.keys())

            # Loop through the datasets to find the one that matches the structure of the Astropy table
            for key in keys:
                data = f[key][:]
                # Check if the data matches the structure of an Astropy table (e.g., 2D array, columns)
                if isinstance(data, (list, np.ndarray)):  # Adjust this check if needed
                    # Assuming that this is the right table structure, convert it into an Astropy table
                    provabgs = Table(data)  # Convert to Astropy Table
                    break
            else:
                raise KeyError(f"No matching dataset found in {file}")

        # Update tf_mastertable with stellar mass information
        tf_mastertable = update_stellar_masses(tf_mastertable, provabgs)

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Optionally, save the updated tf_mastertable after processing all files
tf_mastertable.write('tf_mastertable_massmag_stellar.fits', format='fits', overwrite=True)


Error processing /global/cfs/cdirs/desi/users/chahah/sedflow/desiy1_bgs/BGS_ANY_full.vagc.82.hdf5: arrays used as indices must be of integer (or boolean) type
Error processing /global/cfs/cdirs/desi/users/chahah/sedflow/desiy1_bgs/BGS_ANY_full.vagc.83.hdf5: arrays used as indices must be of integer (or boolean) type
Error processing /global/cfs/cdirs/desi/users/chahah/sedflow/desiy1_bgs/BGS_ANY_full.vagc.68.hdf5: arrays used as indices must be of integer (or boolean) type
Error processing /global/cfs/cdirs/desi/users/chahah/sedflow/desiy1_bgs/BGS_ANY_full.vagc.73.hdf5: arrays used as indices must be of integer (or boolean) type
Error processing /global/cfs/cdirs/desi/users/chahah/sedflow/desiy1_bgs/BGS_ANY_full.vagc.53.hdf5: arrays used as indices must be of integer (or boolean) type
Error processing /global/cfs/cdirs/desi/users/chahah/sedflow/desiy1_bgs/BGS_ANY_full.vagc.0.hdf5: arrays used as indices must be of integer (or boolean) type
Error processing /global/cfs/cdirs/desi/users/c

In [ ]:
# Check common TARGETIDs and their types
provabgs_targetids = provabgs['TARGETID'].data.astype(str)
tf_mastertable_targetids = tf_mastertable['TARGETID'].data.astype(str)
common_targetids = np.intersect1d(provabgs_targetids, tf_mastertable_targetids)

print("Common TARGETIDs:", common_targetids)
print("Number of common TARGETIDs:", len(common_targetids))

# Check if any rows are selected in tf_mastertable
common_rows = tf_mastertable[np.isin(tf_mastertable['TARGETID'], common_targetids)]
print("Filtered rows in tf_mastertable with common TARGETIDs:")
print(common_rows)

# Check the columns you're updating
print("M_STELLAR_PROVABGS values:", tf_mastertable['M_STELLAR_PROVABGS'])
print("M_STELLAR_PROVABGS_ERR values:", tf_mastertable['M_STELLAR_PROVABGS_ERR'])


In [ ]:
'''
# Initialize a counter for skipped files
skipped_count = 0

# Wrap the single file path in a list so the loop iterates correctly
files = ['/global/cfs/cdirs/desi/users/chahah/sedflow/desiy1_bgs/BGS_ANY_full.vagc.83.hdf5']

#Process each file and count how many were skipped
for file_path in files:
    print(f"Processing {file_path}...")
    tf_mastertable, was_skipped = process_file(file_path, tf_mastertable)
    if was_skipped:
        skipped_count += 1

    # Optionally save the updated master table after each iteration
    tf_mastertable.write('intermediate_tf_mastertable.fits', overwrite=True)

# Save the final updated master table
tf_mastertable.write('final_tf_mastertable.fits', overwrite=True)

# Print the total number of skipped files
print(f"Processing complete. Final table saved as 'final_tf_mastertable.fits'.")
print(f"Total number of skipped files: {skipped_count}")

In [12]:
# Assuming 'TARGETID' is the column name in both tables
provabgs_targetids = provabgs['TARGETID'].data  # Extract TARGETID from provabgs table
tf_mastertable_targetids = tf_mastertable['TARGETID'].data  # Extract TARGETID from tf_mastertable table

# Find common TARGETID values
common_targetids = np.intersect1d(provabgs_targetids, tf_mastertable_targetids)

# Optionally, print or return the common TARGETIDs
print("Common TARGETIDs:", common_targetids)

len(common_targetids)

Common TARGETIDs: [39633220072899419 39633220077094831 39633220098064420 39633220106455753
 39633220110650641 39633220114843226 39633220114843482 39633220148396181
 39633220156787159 39633220156787995 39633220198728176 39633220223894583
 39633220278419603 39633220311974884 39633220311976131 39633220311976229
 39633220311977981 39633220320362750 39633220328754273 39633220332947952
 39633220341336736 39633220353919368 39633220366499950 39633220374889492
 39633220374891724 39633220387473365 39633220437804229 39633220442000752
 39633220446193036 39633220454582603 39633220454583584 39633220467164151
 39633220467164474 39633220467166633 39633220479748453 39633220483941323
 39633220500719524 39633220504912922 39633220504913152 39633220504914283
 39633220521689381 39633220521690184 39633220534275497 39633220542663699
 39633220555244993 39633220559438319 39633220567826863 39633220572021914
 39633220572022560 39633220572023215 39633220580410460 39633220580411989
 39633220580412088 39633220580412

638

In [22]:
common_rows = tf_mastertable[np.isin(tf_mastertable['TARGETID'], common_targetids)]

# Print the filtered rows
common_rows


TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,Z_CENTER,ROTCURVE_CHI2,VMAX,VMAX_ERR,RTURN,RTURN_ERR,ALPHA,ALPHA_ERR,INCLINATION,VROT_R26,VROT_R26_ERR,M_R26,M_R26_ERR,VROT_3P5R26,VROT_3P5R26_ERR,M_3P5R26,M_3P5R26_ERR,RABS_MAG26,M_STELLAR_CIGALE,M_STELLAR_CIGALE_ERR,M_STELLAR_PROVABGS,M_STELLAR_PROVABGS_ERR
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39633220098064420,162.038766796335,48.3314519602365,5914,sv1,0.041278770975308,2.28758632187292e-06,0,129189.216787577,fuji/healpix/sv1/bright/59/5914/redrock-sv1-bright-5914.fits,TFT,1061166,162.0387667963351,48.3314519602365,1.0,3.1805546814635168e-15,4.2064617259532895e-13,0.0,0.0,0.13990542991850916,0.8396661454652159,0.04127828503751255,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
39633228151128920,163.824445690751,48.8133284113411,5916,sv1,0.0882250540890508,2.544967089309e-06,0,38958.7898755074,fuji/healpix/sv1/bright/59/5916/redrock-sv1-bright-5916.fits,TFT,186596,163.8244456907507,48.81332841134111,1.0,6.3611093629270335e-15,7.611077614809406e-13,0.0,0.0,-10.333292768177058,0.9242354930846998,0.08826256449128893,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
39633224124597085,162.052614863675,48.5048528495049,5914,sv1,-0.00199569129234795,4.13114935733491e-48,1570,1.94266889222573e+84,fuji/healpix/sv1/bright/59/5914/redrock-sv1-bright-5914.fits,TFT,1093471,162.05261486367482,48.504852849504935,1.0,1.2722218725854067e-14,1.4664243154527452e-12,0.0,0.0,-12505.032710315447,4.655758915588118,0.0414453904495374,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
39633224120403338,161.689096022652,48.5849036766505,5914,sv1,0.0423872080330663,3.4372139052887e-06,0,6254.7524599731,fuji/healpix/sv1/bright/59/5914/redrock-sv1-bright-5914.fits,TFT,1263682,161.6890960226517,48.584903676650534,1.0,0.0,0.0,0.0,0.0,-1.2703169199523363,1.2335537683956952,0.04239162498113517,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
39633232194438753,169.191772769351,49.1174652625242,5611,sv1,0.00712425193211093,3.95376551328423e-05,0,183.722679359984,fuji/healpix/sv1/bright/56/5611/redrock-sv1-bright-5611.fits,TFT,1240422,169.19177276935113,49.117465262524206,1.0,0.0,0.0,0.0,0.0,-9.927673310283307,13.74348188312678,0.007157604110922544,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
39633236212582005,173.702734361046,49.2417345677238,5618,sv2,0.0296302560616445,2.29871102633808e-06,0,6547.37314122915,fuji/healpix/sv2/bright/56/5618/redrock-sv2-bright-5618.fits,TFT,827815,173.702734361046,49.2417345677238,1.0,0.0,0.0,0.0,0.0,0.46830710802617775,0.8111965717523987,0.02962864767423723,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
39633236212582472,173.756742407124,49.213841601924,5618,sv2,0.0367680524637329,2.85154796992167e-06,0,3161.29413351417,fuji/healpix/sv2/bright/56/5618/redrock-sv2-bright-5618.fits,TFT,465634,173.75674240712425,49.21384160192399,1.0,0.0,0.0,0.0,0.0,-0.31158517681693726,1.0352487196259266,0.0367691300154992,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
39633236208389728,173.484516428452,49.3316004094501,5618,sv2,0.0317278856482237,2.87250029928248e-06,0,9232.83814773089,fuji/healpix/sv2/bright/56/5618/redrock-sv2-bright-5618.fits,TFT,7105,173.48451642845188,49.33160040945009,1.0,6.3611093629270335e-15,1.0375813144507129e-12,0.0,0.0,0.4055512750741022,1.0149845867322684,0.031726489956029336,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
39633224162345412,165.397190681015,48.4761022630204,5910,sv1,0.108195984852993,1.14411011896808e-05,0,15409.8101054654

In [26]:
# Find the number of rows where 'M_STELLAR_PROVABGS' > 0
num_rows_with_positive_m_stellar = np.sum(common_rows['M_R26'] > 0)

# Print the result
print("Number of rows where 'M_STELLAR_PROVABGS' > 0:", num_rows_with_positive_m_stellar)


Number of rows where 'M_STELLAR_PROVABGS' > 0: --


In [ ]:
'''
# Filter the table for rows where SURVEY is 'main' and M_STELLAR_PROVABGS > 0
filtered_rows = tf_mastertable[(tf_mastertable['SURVEY'] == 'main') & (tf_mastertable['M_STELLAR_PROVABGS'] > 0)]

# Display the filtered rows
print(filtered_rows)

In [ ]:
#writing new data into table
tf_mastertable.write(data_dir + 'tf_mastertable_massmag_stellar.fits', format='fits', overwrite=True)